<a href="https://colab.research.google.com/github/Juan-Ganan/Analysis-of-Optimization-Algorithms-for-the-TSP-Hill-Climbing-and-Simulated-Annealing./blob/main/SHC_Mejorado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tsplib95

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.4.2
    Uninstalling networkx-3.4.2:
      Successfully uninstalled networkx-3.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.1

In [2]:
import tsplib95
import networkx as nx
import matplotlib.pyplot as plt
import time

In [3]:
def load_tsp(filename):
  '''Carga el problema TSP desde un archivo tsplib'''
  problem = tsplib95.load(filename)
  return problem

def plot_tsp(problem):
  '''Display el grafo del problema TSP con network y matplotlib'''
  G = problem.get_graph()

  if 'node_coords' in problem.as_dict():
    pos = problem.node_coords
  else:
    pos = nx.spring_layout(G)

  plt.figure(figsize=(8, 6))
  nx.draw(G, pos, with_labels=True, node_color='lightblue', edge_color='gray', node_size=30)
  plt.title('Visualización del TSP')
  plt.show()

In [5]:
import networkx as nx
import math

# Función para calcular la distancia euclidiana entre dos puntos (x1, y1) y (x2, y2)
def calcular_distancia(punto1, punto2):
    return math.sqrt((punto1[0] - punto2[0])**2 + (punto1[1] - punto2[1])**2)

# Crear un grafo vacío
problemHC = nx.Graph()

# Abrir y leer el archivo TSP
with open('att48.tsp', 'r') as file:
    lineas = file.readlines()

    # Leer las coordenadas de los nodos
    puntos = {}
    leyendo = False
    for linea in lineas:
        if linea.startswith("NODE_COORD_SECTION"):
            leyendo = True
        elif linea.startswith("EOF"):
            break
        elif leyendo:
            datos = linea.split()
            nodo = int(datos[0])
            x = float(datos[1])
            y = float(datos[2])
            puntos[nodo] = (x, y)

# Agregar los nodos y las aristas al grafo
for nodo1, coord1 in puntos.items():
    for nodo2, coord2 in puntos.items():
        if nodo1 != nodo2:  # Evitar la auto-conexión
            distancia = calcular_distancia(coord1, coord2)
            problemHC.add_edge(nodo1, nodo2, weight=distancia)

In [35]:
import random
import math
import networkx as nx

def calcular_costo(recorrido, grafo):
    costo = 0
    costo = sum(grafo[recorrido[i]][recorrido[i+1]]['weight'] for i in range(len(recorrido)-1))
    costo += grafo[recorrido[-1]][recorrido[0]]['weight']
    return costo


def generar_vecino(recorrido):
    i, j = sorted(random.sample(range(len(recorrido)), 2))
    nuevo_recorrido = recorrido[:i] + recorrido[i:j+1][::-1] + recorrido[j+1:]
    return nuevo_recorrido

def simulated_annealing_tsp(grafo, temp_inicial=1000, temp_final=1, factor_enfriamiento=0.995, max_iteraciones=1000):
    nodos = list(grafo.nodes())
    recorrido_actual = random.sample(nodos, len(nodos))
    costo_actual = calcular_costo(recorrido_actual, grafo)
    history = [costo_actual]
    temperatura = temp_inicial

    while temperatura > temp_final:
        iteraciones_por_temperatura = max_iteraciones * temperatura / temp_inicial

        for _ in range(int(iteraciones_por_temperatura)):
            vecino = generar_vecino(recorrido_actual)
            costo_vecino = calcular_costo(vecino, grafo)
            delta = costo_vecino - costo_actual
            if delta < 0 or random.random() < math.exp(-delta / temperatura):
                recorrido_actual = vecino
                costo_actual = costo_vecino
            history.append(costo_actual)

        temperatura *= factor_enfriamiento

    return recorrido_actual, costo_actual, history

In [44]:
problem = load_tsp('att48.tsp')
# Get networkx graph from problem
graph = problem.get_graph()
# Get node coordinates if available
if 'node_coords' in problem.as_dict():
  puntos = problem.node_coords
else:
  puntos = nx.spring_layout(graph)
recorrido, costo, history = simulated_annealing_tsp(graph)
print("Mejor recorrido:", recorrido)
print("Costo del recorrido:", costo)

Mejor recorrido: [43, 17, 27, 19, 37, 6, 28, 7, 18, 44, 31, 38, 9, 8, 1, 22, 16, 3, 23, 13, 25, 14, 34, 41, 29, 2, 26, 4, 35, 45, 10, 24, 42, 5, 48, 39, 32, 21, 47, 11, 40, 15, 12, 20, 33, 46, 36, 30]
Costo del recorrido: 10797


In [38]:
import random
import math
import networkx as nx

def calcular_costo_incremental(recorrido, grafo, i=None, j=None):

    costo = 0
    if i is None or j is None:
        costo = sum(grafo[recorrido[i]][recorrido[i+1]]['weight'] for i in range(len(recorrido)-1))
        costo += grafo[recorrido[-1]][recorrido[0]]['weight']  # Costo de vuelta al inicio
    else:

        costo -= grafo[recorrido[i-1]][recorrido[i]]['weight']  # Quitamos el antiguo costo
        costo -= grafo[recorrido[j]][recorrido[(j+1) % len(recorrido)]]['weight']
        recorrido[i:j+1] = recorrido[i:j+1][::-1]  # Revertimos el segmento
        costo += grafo[recorrido[i-1]][recorrido[i]]['weight']  # Sumamos el nuevo costo
        costo += grafo[recorrido[j]][recorrido[(j+1) % len(recorrido)]]['weight']
    return costo

def generar_vecino(recorrido):
    i, j = sorted(random.sample(range(len(recorrido)), 2))
    return recorrido[:i] + recorrido[i:j+1][::-1] + recorrido[j+1:]

# Hill Climbing mejorado para TSP
def hill_climbing_tsp(grafo, max_iteraciones=100000):
    nodos = list(grafo.nodes())
    recorrido_actual = random.sample(nodos, len(nodos))
    costo_actual = calcular_costo_incremental(recorrido_actual, grafo) # Cálculo inicial optimizado
    history = [costo_actual]

    for _ in range(max_iteraciones):
        vecino = generar_vecino(recorrido_actual)
        costo_vecino = calcular_costo_incremental(vecino, grafo)

        if costo_vecino < costo_actual:
            recorrido_actual = vecino
            costo_actual = costo_vecino
        history.append(costo_actual)

    return recorrido_actual, costo_actual, history


In [45]:
problem = load_tsp('att48.tsp')

graph = problem.get_graph()

if 'node_coords' in problem.as_dict():
  puntos = problem.node_coords
else:
  puntos = nx.spring_layout(graph)
recorrido, costo, history = hill_climbing_tsp(graph)
print("Mejor recorrido:", recorrido)
print("Costo del recorrido:", costo)

Mejor recorrido: [5, 48, 39, 25, 14, 13, 12, 11, 23, 3, 34, 41, 16, 22, 1, 8, 9, 40, 15, 46, 38, 31, 44, 18, 7, 28, 6, 37, 19, 27, 17, 43, 30, 36, 33, 20, 47, 21, 32, 24, 10, 45, 35, 4, 26, 42, 2, 29]
Costo del recorrido: 11199
